📊 Stats to Explore for GW1
1. 🔥 Top Points Per Game(PPG) Players from 2024/25
2. 💰 Best Value Players (Points per Million)
3. 🛡️ Defenders with High Returns
4. ⚠️ Injured or Flagged Players
5. 📅 Fixture Difficulty (Team-Level)

In [8]:
import json
import pandas as pd
import os

# Get the current working directory and construct the correct path
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Use absolute path to the data file
project_root = os.path.dirname(current_dir)
data_file_path = os.path.join(project_root, 'data', 'bootstrap_static.json')

with open(data_file_path) as f:
    data = json.load(f)

players_df = pd.DataFrame(data['elements'])
teams_df = pd.DataFrame(data['teams'])

print(f"Loaded {len(players_df)} players and {len(teams_df)} teams")

# Example: Top 15 players by most points per game
most_ppg = players_df[['web_name', 'points_per_game', 'now_cost', 'minutes']].sort_values(by='points_per_game', ascending=False).head(15)

# Example: Top 15 players by points per million
players_df['points_per_million'] = players_df['total_points'] / (players_df['now_cost'] / 10)

points_per_million = players_df[['web_name', 'total_points', 'now_cost', 'points_per_million']].sort_values(by='points_per_million', ascending=False).head(15)

# Example: Top 10 defenders by total points
# Note: 'element_type' 2 corresponds to defenders in the FPL data
defenders = players_df[players_df['element_type'] == 2] 
top_defenders = defenders[['web_name', 'total_points', 'clean_sheets', 'goals_scored', 'assists', 'bonus', 'now_cost']].sort_values(by='total_points', ascending=False).head(10)

# Example: Players with injury updates and their chance of playing this round
flagged = players_df[players_df['news'].str.len() > 0]
flagged_players = flagged[['web_name', 'news', 'chance_of_playing_this_round']]

# Fixture difficulty for each team (first 3-5 fixtures)
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_df = pd.read_json(fixtures_url)

# Only keep fixtures from gameweek 1 to 5
early_fixtures = fixtures_df[fixtures_df['event'] <= 5]

# Count easy fixtures (difficulty <= 2) for each team
easy_home_fixtures = early_fixtures[early_fixtures['team_h_difficulty'] <= 2]['team_h'].value_counts()
easy_away_fixtures = early_fixtures[early_fixtures['team_a_difficulty'] <= 2]['team_a'].value_counts()

easy_fixtures = (easy_home_fixtures + easy_away_fixtures).sort_values(ascending=False)

print("Analysis complete! Check the variables above for results.")

# Most defensive contributions by defender
defenders_defensive_contributions = defenders[['web_name', 'defensive_contribution', 'clearances_blocks_interceptions', 'tackles']].sort_values(by='defensive_contribution', ascending=False).head(20)

Current working directory: c:\Users\Brian\Documents\Projects\fpl-analyser\notebooks
Loaded 719 players and 20 teams
Analysis complete! Check the variables above for results.


In [9]:
# Display the analysis results

print("🔥 TOP 15 PLAYERS BY POINTS PER GAME:")
print("=" * 50)
print(most_ppg.to_string(index=False))

print("\n\n💰 TOP 15 PLAYERS BY POINTS PER MILLION:")
print("=" * 50)
print(points_per_million.to_string(index=False))

print("\n\n🛡️ TOP 10 DEFENDERS BY TOTAL POINTS:")
print("=" * 50)
print(top_defenders.to_string(index=False))

print("\n\n⚠️ PLAYERS WITH INJURY/NEWS UPDATES:")
print("=" * 50)
if len(flagged_players) > 0:
    print(flagged_players.to_string(index=False))
else:
    print("No players currently flagged with injury concerns!")

print("\n\n📅 TEAMS WITH MOST EASY FIXTURES (GW1-5):")
print("=" * 50)
print(easy_fixtures.head(10).to_string())

print("\n\n🛡️ TOP 20 DEFENDERS BY DEFENSIVE CONTRIBUTIONS:")
print("=" * 50)
print(defenders_defensive_contributions.to_string(index=False))

🔥 TOP 15 PLAYERS BY POINTS PER GAME:
  web_name points_per_game  now_cost  minutes
 Calafiori             9.3        56      224
Acheampong             9.0        40       90
   Ballard             9.0        46       98
  Chalobah             9.0        51      248
  J.Timber             8.7        56      172
João Pedro             8.7        77      252
      Enzo             8.3        66      258
    Senesi             8.3        45      270
   Haaland             8.0       141      252
    Botman             8.0        49       67
   Semenyo             7.7        74      270
     Guéhi             7.7        45      270
   O.Dango             7.5        60      173
  Grealish             7.3        67      196
   Ekitiké             7.3        87      228


💰 TOP 15 PLAYERS BY POINTS PER MILLION:
   web_name  total_points  now_cost  points_per_million
     Senesi            25        45            5.555556
   Chalobah            27        51            5.294118
      Guéhi      

Player-vs-Player Comparison Tool

In [24]:
# Function to compare players based on their names
# This function takes a list of player names and returns their stats for comparison
# Example usage: compare_players(['N. Williams', 'Aina'])
# Note: Ensure that the player names are exactly as they appear in the 'web_name'
def compare_players(names):
    cols = ['web_name', 'minutes', 'total_points', 'points_per_million', 'points_per_game',
            'now_cost', 'clean_sheets', 'goals_scored', 'assists', 'bonus']
    comparison = players_df[players_df['web_name'].isin(names)][cols]
    return comparison
print(compare_players(['N.Williams', 'Van de Ven', 'Aina']))


       web_name  minutes  total_points  points_per_million points_per_game  \
515        Aina     2996           128           25.600000             3.7   
516  N.Williams     2588           109           21.800000             3.1   
586  Van de Ven     1014            32            7.111111             2.5   

     now_cost  clean_sheets  goals_scored  assists  bonus  
515        50            12             2        1     15  
516        50            10             1        3      8  
586        45             2             0        2      2  
